### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nanonets/Nanonets-OCR-s',
 '/MiniMaxAI/MiniMax-M1-80k',
 '/google/magenta-realtime',
 '/mistralai/Mistral-Small-3.2-24B-Instruct-2506',
 '/Menlo/Jan-nano',
 '/models',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/enzostvs/deepsite',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/MiniMaxAI/MiniMax-M1',
 '/spaces/nvidia/PartPacker',
 '/spaces',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/AceReason-1.1-SFT',
 '/datasets/nvidia/Nemotron-Personas',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers'

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps


In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnanonets/Nanonets-OCR-s\nUpdated\n4 days ago\n•\n177k\n•\n1.1k\nMiniMaxAI/MiniMax-M1-80k\nUpdated\nabout 3 hours ago\n•\n10.4k\n•\n540\ngoogle/magenta-realtime\nUpdated\n1 day ago\n•\n251\nmistralai/Mistral-Small-3.2-24B-Instruct-2506\nUpdated\n2 days ago\n•\n5.37k\n•\n206\nMenlo/Jan-nano\nUpdated\n7 days ago\n•\n29.1k\n•\n385\nBrowse 1M+ models\nSpaces\nRunning\n744\n744\nSparc3D\n🏃\nNext

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

**The AI community building the future.**

At Hugging Face, we empower the machine learning community to **create, discover, and collaborate** on innovative models, datasets, and applications. With over **1 million models** and **250,000 datasets** hosted on our platform, we are the cornerstone of modern AI development.

---

## Our Offerings

### **Models**
Explore our vast repository of machine learning models, from OCR to advanced generative applications. Trending models are updated frequently to ensure the latest advancements are always available.

### **Datasets**
Hugging Face hosts a plethora of datasets for various ML tasks. Whether you are working on research, production, or a hobby project, our datasets serve as essential resources.

### **Spaces**
Our collaborative environment allows you to deploy and manage applications seamlessly. Engage with cutting-edge AI applications like 3D model generation or code generation directly.

### **Enterprise Solutions**
Join over **50,000 organizations** using Hugging Face, including industry leaders like **Google, Microsoft, and Amazon**. Our Enterprise plans provide advanced security features, dedicated support, and access controls tailored for organizations looking to thrive in the AI landscape.

---

## Company Culture

### **Community-Driven**
At the heart of Hugging Face is an ethos of community and collaboration. We believe that the best tools are built by the community for the community. Our open-source philosophy not only drives innovation but also fosters a sense of belonging among members.

### **Inclusive Environment**
We are committed to creating a culture that values diversity and inclusion. Our team thrives on collaboration and innovation, welcoming people from varied backgrounds and experiences.

### **Continuous Learning**
Hugging Face encourages a culture of continuous learning and growth. We support our employees' professional development and provide opportunities to explore new technologies and practices in the rapidly evolving field of AI.

---

## Careers at Hugging Face

Join us in building the future of AI! We are constantly on the lookout for passionate, talented individuals who want to make a difference in the world of machine learning. Whether you are an engineer, researcher, or contribute to community outreach, there’s a place for you at Hugging Face.

### **Current Openings**
We offer a variety of roles across different functions. Visit our [jobs page](link to jobs page) to explore opportunities and see how you can contribute to our mission.

---

## Connect With Us

Join our vibrant community on platforms like Discord, GitHub, and LinkedIn. Stay updated with our latest developments by following us on [Twitter](link to Twitter page) and browsing our extensive [Docs](link to documentation).

**Together, let's shape the future of AI. Welcome to Hugging Face!**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Company Brochure

## Overview

**Hugging Face** is a pioneering company in the realm of artificial intelligence, dedicated to building the future with the AI community. With a state-of-the-art platform for models, datasets, and applications, Hugging Face acts as a collaborative hub for machine learning enthusiasts, researchers, and businesses.

## Our Mission

At Hugging Face, we are on a mission to democratize access to machine learning tools and knowledge. We empower individuals and organizations to create, discover, and collaborate on machine learning projects, enabling advancements in AI technologies.

## Key Offerings

1. **Models**: Explore a rich library of over 1 million models, including state-of-the-art machine learning implementations for various applications.

2. **Datasets**: Access and share a vast collection of over 250,000 datasets, providing essential resources for diverse machine learning tasks.

3. **Spaces**: A platform for hosting and collaborating on applications, with over 400,000 applications available, fostering community-driven innovation.

4. **Enterprise Solutions**: Our enterprise offerings ensure organizations can build AI with advanced security, access control, and dedicated support. Starting at $20/user/month, we provide robust platforms for organizations to leverage AI effectively.

## Community and Culture

At Hugging Face, we believe in the power of collaboration and openness. Our community comprises over 50,000 organizations, including notable names like Google, Amazon, and Microsoft. Here, contributors from diverse backgrounds come together to innovate and share knowledge, making machine learning more accessible.

Our culture is centered around inclusivity and empathy, embracing diverse perspectives. As part of the Hugging Face team, you will engage in meaningful projects that have a real impact on the AI landscape while fostering personal growth and development.

## Career Opportunities

We are always looking for passionate individuals to join our mission! Opportunities at Hugging Face span various roles such as:

- Machine Learning Engineers
- Data Scientists
- Community Managers
- Product Managers

We value teamwork, creativity, and innovation, making Hugging Face a great place for those who want to make a difference while working in a supportive environment. If you are interested in joining our team, check out our [Jobs Page](https://huggingface.co/jobs).

## Join Us!

Whether you're a developer, researcher, or simply an AI enthusiast, Hugging Face welcomes you to explore, contribute, and grow with us. Sign up today to start your journey in the world of machine learning!

- **Website:** [huggingface.co](https://huggingface.co)
- **Follow Us:** Join our community on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), and [GitHub](https://github.com/huggingface) for the latest updates.

Be a part of shaping the future of AI with Hugging Face!

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

### The AI Community Building the Future

At Hugging Face, we are dedicated to creating a collaborative space where machine learning enthusiasts, researchers, and companies come together to innovate and develop artificial intelligence solutions. Our mission is simple: democratize machine learning and enhance the accessibility of cutting-edge technologies.

---

## What We Offer

- **Models**: Browse over 1 million models crafted by the community, ranging from text generation to image processing and beyond.
  
- **Datasets**: Access over 250,000 datasets critical for training and evaluating machine learning models.
  
- **Spaces**: Collaborate on projects in real-time with our user-friendly applications, including next-gen 3D model generation and code generation from text prompts.
  
- **Enterprise Solutions**: Our platform supports over 50,000 organizations with enterprise-grade security, dedicated support, model hosting, and developer incentives.

---

## Our Community 

At Hugging Face, we believe in the power of community collaboration. Our platform is frequented by:

- **Top Enterprises**: Trusted by industry leaders including Google, Microsoft, Meta, and Amazon, with thousands of models and a supportive partnership.
  
- **Contributors & Innovators**: Our active user base contributes continuously to our ecosystem, resulting in a rich collection of state-of-the-art Machine Learning tools such as Transformers, Diffusers, and Tokenizers.

---

### Culture

We nurture a culture of inclusivity, creativity, and growth. Our team comprises passionate individuals committed to shaping the future of AI. We encourage collaborative problem-solving and continuous learning to foster innovation.

- **Open Source Philosophy**: Our projects are open-source, allowing everyone to contribute and leverage community knowledge.
  
- **Diverse Workforce**: We welcome talent from all backgrounds, believing that diversity drives better ideas and innovation.

---

## Careers with Hugging Face

Join us in our mission to democratize machine learning! We are looking for skilled professionals who are passionate about AI and eager to contribute to our community.

- **Current Opportunities**: Explore various roles in engineering, product management, marketing, and community engagement on our [Jobs Page](https://huggingface.co/jobs).
  
- **Work Environment**: We offer a flexible work environment that values the work-life balance and the professional growth of our employees.

---

### Connect with Us

Join the Hugging Face community today to explore, create, and collaborate on transformative AI solutions. 

- **Website**: [huggingface.co](https://huggingface.co)  
- **Follow Us**: [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), [Discord](https://discord.gg/huggingface)  

Together, let's build the future of AI!

# Business applications
Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!